In [1]:
import random
import csv
import numpy as np
def loader(file):
    with open(file) as data:
        data_raw = list(csv.reader(data))
    return data_raw
raw_data = loader("sonar.all-data")
for row in raw_data:
    for i in row[-1]:
        if i == "R" or i == "M":
            key = row.pop(-1)
            if key == "M":
                key = 1
            elif key == "R":
                key = 0
            row.append(key)
for row in raw_data:
    for i in row:
        i == float(i)
dataset = [list(map(float, row)) for row in raw_data]
###### trsize = size of train set (from 0 to 1)
def split(dataset, trsize):
    train_set = []
    test_set = []
    random.shuffle(dataset)
    for i in range(int(len(dataset)*trsize)):
        train_set.append(dataset[i])
    for i in range(int(len(dataset)*(1-trsize))):
        test_set.append(dataset[i])
    return train_set, test_set
        
train_set, test_set = split(dataset, 0.8)

In [2]:
weights = np.zeros(len(train_set[0])-1)

In [3]:
def predict(train, epoch_n, weights, lrate):
    z = 0
    for i in range(epoch_n):
        predictions = []
        for row in train:
            sum = np.dot(weights, row[:-1])
            predicted = 0.0 if sum<z else 1.0
            predictions.append(predicted)
            for n in range(len(weights)):
                weights[n] += lrate*(row[-1]-predicted)*row[n]
    return predictions
predictions = predict(train_set, 5000, weights, 0.1)
def accuracy(predicted, train):
    score = 0
    actual = [row[-1] for row in train]
    for i in range(len(predicted)):
        if predicted[i] == actual[i]:
            score += 1
    return score/float(len(predicted))*100

def test(test, weights):
    tpred = []
    tactual = [row[-1] for row in test]
    for row in test_set:
        pred = np.dot(row[:-1], weights) 
        tpred.append(0 if pred<0 else 1)
        
    return tpred, tactual
tpred, tactual = test(test_set, weights)

In [4]:
def evaluation(tpred, tacutal):
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    for i in range(len(tactual)):
        if tpred[i] == tactual[i]:
            if tpred[i] == 0:
                tn += 1
            else:
                tp += 1
        else:
            if tpred[i] == 0:
                fn += 1
            else:
                fp += 1
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    accuracy = (tp+tn)/(tp+tn+fn+fp)
    f1 = 2*((precision*recall)/(precision+recall))
    return print(f"""Accuracy:  {accuracy}
Precision: {precision}
Recall:    {recall}
F1 Score:  {f1}""")


evaluation(tpred, tactual)    

Accuracy:  0.8780487804878049
Precision: 0.8148148148148148
Recall:    1.0
F1 Score:  0.8979591836734693
